### Combining NAAF-corrected *de novo* and PeaksDB  peptides for Trocas 7 incubation samples

The dataset:

    16 treatments: 4 stations, 2 timepoints (Time 0 an Time 24 hrs), 2 size fractions (GFF; GF75):
    
    Stations: 
    
        - Macapa South (MS) South stem, upriver
        - Macapa North (MN) North stem, upriver
        - Chaves (CV) South stem, downriver
        - Baylique (BY) North stem, downriver


    Proteomics samples from 2 trips to UWPR (July 2020 on the QE; April 2021 on the Fusion)
    There were at least triplicate samples for each treatment
    Many in April 2021 injected twice

Starting with:

    Peaks de novo results of PTM-optimized sequencing, NAAF corrected per sample
    PeaksDB de novo-assisted results from PTM-optimized database searches, NAAF corrected per sample
    
    Multiple samples per treatment

Goal:

    CSVs with combined de novo and PeaksDB for each sample, normalized to by NAAF
    
Using:

    - pandas
    - matplotlib
    - numpy

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations


### 1. Chaves, timepoint 0, size fraction 0.3-0.7 um (denoted as CV_T00_GF)
### T7 samples #s: 105 (106C), 106A, 106B
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/106_CV_T00_GF_PDB
    Trocas-incubations/processed/PeaksDB/106_CV_T00_GF_DN

In [3]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_106a = pd.read_csv("processed/PeaksDN/106_CV_T00_GF_DN/106A_CV_T00_GF_DN50_naaf_totals.csv")
peaks50_106b = pd.read_csv("processed/PeaksDN/106_CV_T00_GF_DN/106B_CV_T00_GF_DN50_naaf_totals.csv")
peaks50_106c = pd.read_csv("processed/PeaksDN/106_CV_T00_GF_DN/106C_CV_T00_GF_DN50_naaf_totals.csv")
peaksdb_106a = pd.read_csv("processed/PeaksDB/106_CV_T00_GF_PDB/106A_CV_T00_GF_PDB_naaf_totals.csv")
peaksdb_106b = pd.read_csv("processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_naaf_totals.csv")
peaksdb_106c = pd.read_csv("processed/PeaksDB/106_CV_T00_GF_PDB/106C_CV_T00_GF_PDB_naaf_totals.csv")

frames = [peaks50_106a, peaks50_106b, peaks50_106c, peaksdb_106a, peaksdb_106b, peaksdb_106c]
index = ['peaks50_106a', 'peaks50_106b', 'peaks50_106c', 'peaksdb_106a', 'peaksdb_106b', 'peaksdb_106c']

# concatenate dataframes
tot_106 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_106 = tot_106.loc[:, ~tot_106.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_106a', 'peaks50_106b', 'peaks50_105', 'peaksdb_106a', 'peaksdb_106b', 'peaksdb_105']
tot_106.insert(loc=0, column='data source', value=names)
tot_106.set_index('data source')

# sum the AAs and PTMN

index = ['106']

data = {'NAAF': tot_106['NAAF'].sum(),
        'A-NAAF': tot_106['A'].sum(),
        'C-NAAF': tot_106['C'].sum(),
        'D-NAAF': tot_106['D'].sum(),
        'E-NAAF': tot_106['E'].sum(),
        'F-NAAF': tot_106['F'].sum(),
        'G-NAAF': tot_106['G'].sum(),
        'H-NAAF': tot_106['H'].sum(),
        'I-NAAF': tot_106['I'].sum(),
        'K-NAAF': tot_106['K'].sum(),
        'L-NAAF': tot_106['L'].sum(),
        'M-NAAF': tot_106['M'].sum(),
        'N-NAAF': tot_106['N'].sum(),
        'P-NAAF': tot_106['P'].sum(),
        'Q-NAAF': tot_106['Q'].sum(),
        'R-NAAF': tot_106['R'].sum(),
        'S-NAAF': tot_106['S'].sum(),
        'T-NAAF': tot_106['T'].sum(),
        'V-NAAF': tot_106['V'].sum(),
        'W-NAAF': tot_106['W'].sum(),
        'Y-NAAF': tot_106['Y'].sum(),
        'C-carb-NAAF': tot_106['c-carb'].sum(),
        'M-oxid-NAAF': tot_106['m-oxid'].sum(),
        'N-deam-NAAF': tot_106['n-deam'].sum(),
        'Q-deam-NAAF': tot_106['q-deam'].sum()
       }

sum_106 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_106['AA-NAAF'] = sum_106['A-NAAF'] + sum_106['C-NAAF'] + sum_106['D-NAAF'] + sum_106['E-NAAF'] + \
                     sum_106['F-NAAF'] + sum_106['G-NAAF'] + sum_106['H-NAAF'] + sum_106['I-NAAF'] + \
                     sum_106['K-NAAF'] + sum_106['L-NAAF'] + sum_106['M-NAAF'] + sum_106['N-NAAF'] + \
                     sum_106['P-NAAF'] + sum_106['Q-NAAF'] + sum_106['R-NAAF'] + sum_106['S-NAAF'] + \
                     sum_106['T-NAAF'] + sum_106['V-NAAF'] + sum_106['W-NAAF'] + sum_106['Y-NAAF'] 

sum_106['AA-modifiable-NAAF'] = sum_106['M-NAAF'] + sum_106['N-NAAF'] + sum_106['Q-NAAF']
                                

sum_106['Cys. w/ carb.'] = sum_106['C-carb-NAAF'] / sum_106['C-NAAF'] #1
sum_106['Met. w/ oxid.'] = sum_106['M-oxid-NAAF'] / sum_106['M-NAAF'] #2
sum_106['Asn. w/ deam.'] = sum_106['N-deam-NAAF'] / sum_106['N-NAAF'] #3
sum_106['Glu. w/ deam.'] = sum_106['Q-deam-NAAF'] / sum_106['Q-NAAF'] #3


sum_106['Overall modified'] = 0.001*((sum_106['Met. w/ oxid.']*sum_106['M-NAAF']) + \
                                     (sum_106['Asn. w/ deam.']*sum_106['N-NAAF']) + \
                                     (sum_106['Glu. w/ deam.']*sum_106['Q-NAAF']) / (sum_106['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_106.to_csv("processed/NAAF-sums/NAAF-sum-106.csv")

sum_106.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
106,1242.484063,919.835,140.726813,62.356207,86.20904,76.410784,146.990298,139.292689,30.748749,160.096489,...,48.71591,36.152429,11.47467,10932.535721,422.427593,1.0,0.356624,0.154402,0.222035,0.084896


### 2. Chaves, timepoint 0, size fraction>0.7 um (denoted as CV_T00_GD)
### T7 samples #s: 206
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/206_CV_T00_GD_PDB
    Trocas-incubations/processed/PeaksDB/206_CV_T00_GD_DN

In [4]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_206a = pd.read_csv("processed/PeaksDN/206_CV_T00_GD_DN/206_CV_T00_GD_DN50_naaf_totals.csv")
#peaks50_206b = pd.read_csv("processed/PeaksDN/206_CV_T00_GD_DN/206B_CV_T00_GD_DN50_naaf_totals.csv")
peaksdb_206a = pd.read_csv("processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_naaf_totals.csv")
#peaksdb_206b = pd.read_csv("processed/PeaksDB/206_CV_T00_GD_PDB/206B_CV_T00_GD_PDB_naaf_totals.csv")

frames = [peaks50_206a, peaksdb_206a]
index = ['peaks50_206a', 'peaksdb_206a']

# concatenate dataframes
tot_206 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_206 = tot_206.loc[:, ~tot_206.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_206', 'peaksdb_206']
tot_206.insert(loc=0, column='data source', value=names)
tot_206.set_index('data source')

# sum the AAs and PTMN

index = ['206']

data = {'NAAF': tot_206['NAAF'].sum(),
        'A-NAAF': tot_206['A'].sum(),
        'C-NAAF': tot_206['C'].sum(),
        'D-NAAF': tot_206['D'].sum(),
        'E-NAAF': tot_206['E'].sum(),
        'F-NAAF': tot_206['F'].sum(),
        'G-NAAF': tot_206['G'].sum(),
        'H-NAAF': tot_206['H'].sum(),
        'I-NAAF': tot_206['I'].sum(),
        'K-NAAF': tot_206['K'].sum(),
        'L-NAAF': tot_206['L'].sum(),
        'M-NAAF': tot_206['M'].sum(),
        'N-NAAF': tot_206['N'].sum(),
        'P-NAAF': tot_206['P'].sum(),
        'Q-NAAF': tot_206['Q'].sum(),
        'R-NAAF': tot_206['R'].sum(),
        'S-NAAF': tot_206['S'].sum(),
        'T-NAAF': tot_206['T'].sum(),
        'V-NAAF': tot_206['V'].sum(),
        'W-NAAF': tot_206['W'].sum(),
        'Y-NAAF': tot_206['Y'].sum(),
        'C-carb-NAAF': tot_206['c-carb'].sum(),
        'M-oxid-NAAF': tot_206['m-oxid'].sum(),
        'N-deam-NAAF': tot_206['n-deam'].sum(),
        'Q-deam-NAAF': tot_206['q-deam'].sum()
       }

sum_206 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_206['AA-NAAF'] = sum_206['A-NAAF'] + sum_206['C-NAAF'] + sum_206['D-NAAF'] + sum_206['E-NAAF'] + \
                     sum_206['F-NAAF'] + sum_206['G-NAAF'] + sum_206['H-NAAF'] + sum_206['I-NAAF'] + \
                     sum_206['K-NAAF'] + sum_206['L-NAAF'] + sum_206['M-NAAF'] + sum_206['N-NAAF'] + \
                     sum_206['P-NAAF'] + sum_206['Q-NAAF'] + sum_206['R-NAAF'] + sum_206['S-NAAF'] + \
                     sum_206['T-NAAF'] + sum_206['V-NAAF'] + sum_206['W-NAAF'] + sum_206['Y-NAAF'] 

sum_206['AA-modifiable-NAAF'] = sum_206['M-NAAF'] + sum_206['N-NAAF'] + sum_206['Q-NAAF']
                                

sum_206['Cys. w/ carb.'] = sum_206['C-carb-NAAF'] / sum_206['C-NAAF'] #1
sum_206['Met. w/ oxid.'] = sum_206['M-oxid-NAAF'] / sum_206['M-NAAF'] #2
sum_206['Asn. w/ deam.'] = sum_206['N-deam-NAAF'] / sum_206['N-NAAF'] #3
sum_206['Glu. w/ deam.'] = sum_206['Q-deam-NAAF'] / sum_206['Q-NAAF'] #3


sum_206['Overall modified'] = 0.001*((sum_206['Met. w/ oxid.']*sum_206['M-NAAF']) + \
                                     (sum_206['Asn. w/ deam.']*sum_206['N-NAAF']) + \
                                     (sum_206['Glu. w/ deam.']*sum_206['Q-NAAF']) / (sum_206['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_206.to_csv("processed/NAAF-sums/NAAF-sum-206.csv")

sum_206.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
206,454.580708,322.155898,7.914108,22.869548,33.710607,22.692232,39.109161,29.029865,18.463569,18.84087,...,3.954694,7.204314,3.374535,3944.72973,135.916535,1.0,0.258526,0.068865,0.210845,0.011184


### 3. Chaves, timepoint 24 hrs, size fraction 0.3-0.7 um (denoted as CV_T24_GF)
### T7 samples #s: 306A, 306B, 307 (306C)
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/306_CV_T24_GF_PDB
    Trocas-incubations/processed/PeaksDB/306_CV_T24_GF_DN

In [5]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_306a = pd.read_csv("processed/PeaksDN/306_CV_T24_GF_DN/306A_CV_T24_GF_DN50_naaf_totals.csv")
peaks50_306b = pd.read_csv("processed/PeaksDN/306_CV_T24_GF_DN/306B_CV_T24_GF_DN50_naaf_totals.csv")
peaks50_306c = pd.read_csv("processed/PeaksDN/306_CV_T24_GF_DN/306C_CV_T24_GF_DN50_naaf_totals.csv")
peaksdb_306a = pd.read_csv("processed/PeaksDB/306_CV_T24_GF_PDB/306A_CV_T24_GF_PDB_naaf_totals.csv")
peaksdb_306b = pd.read_csv("processed/PeaksDB/306_CV_T24_GF_PDB/306B_CV_T24_GF_PDB_naaf_totals.csv")
peaksdb_306c = pd.read_csv("processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_naaf_totals.csv")

frames = [peaks50_306a, peaks50_306b, peaks50_306c, peaksdb_306a, peaksdb_306b, peaksdb_306c]
index = ['peaks50_306a', 'peaks50_306b', 'peaks50_306c', 'peaksdb_306a', 'peaksdb_306b', 'peaksdb_306bc']

# concatenate dataframes
tot_306 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_306 = tot_306.loc[:, ~tot_306.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_306a', 'peaks50_306b', 'peaks50_307', 'peaksdb_306a', 'peaksdb_306b', 'peaksdb_307']
tot_306.insert(loc=0, column='data source', value=names)
tot_306.set_index('data source')

# sum the AAs and PTMN

index = ['306']

data = {'NAAF': tot_306['NAAF'].sum(),
        'A-NAAF': tot_306['A'].sum(),
        'C-NAAF': tot_306['C'].sum(),
        'D-NAAF': tot_306['D'].sum(),
        'E-NAAF': tot_306['E'].sum(),
        'F-NAAF': tot_306['F'].sum(),
        'G-NAAF': tot_306['G'].sum(),
        'H-NAAF': tot_306['H'].sum(),
        'I-NAAF': tot_306['I'].sum(),
        'K-NAAF': tot_306['K'].sum(),
        'L-NAAF': tot_306['L'].sum(),
        'M-NAAF': tot_306['M'].sum(),
        'N-NAAF': tot_306['N'].sum(),
        'P-NAAF': tot_306['P'].sum(),
        'Q-NAAF': tot_306['Q'].sum(),
        'R-NAAF': tot_306['R'].sum(),
        'S-NAAF': tot_306['S'].sum(),
        'T-NAAF': tot_306['T'].sum(),
        'V-NAAF': tot_306['V'].sum(),
        'W-NAAF': tot_306['W'].sum(),
        'Y-NAAF': tot_306['Y'].sum(),
        'C-carb-NAAF': tot_306['c-carb'].sum(),
        'M-oxid-NAAF': tot_306['m-oxid'].sum(),
        'N-deam-NAAF': tot_306['n-deam'].sum(),
        'Q-deam-NAAF': tot_306['q-deam'].sum()
       }

sum_306 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_306['AA-NAAF'] = sum_306['A-NAAF'] + sum_306['C-NAAF'] + sum_306['D-NAAF'] + sum_306['E-NAAF'] + \
                     sum_306['F-NAAF'] + sum_306['G-NAAF'] + sum_306['H-NAAF'] + sum_306['I-NAAF'] + \
                     sum_306['K-NAAF'] + sum_306['L-NAAF'] + sum_306['M-NAAF'] + sum_306['N-NAAF'] + \
                     sum_306['P-NAAF'] + sum_306['Q-NAAF'] + sum_306['R-NAAF'] + sum_306['S-NAAF'] + \
                     sum_306['T-NAAF'] + sum_306['V-NAAF'] + sum_306['W-NAAF'] + sum_306['Y-NAAF'] 

sum_306['AA-modifiable-NAAF'] = sum_306['M-NAAF'] + sum_306['N-NAAF'] + sum_306['Q-NAAF']
                                

sum_306['Cys. w/ carb.'] = sum_306['C-carb-NAAF'] / sum_306['C-NAAF'] #1
sum_306['Met. w/ oxid.'] = sum_306['M-oxid-NAAF'] / sum_306['M-NAAF'] #2
sum_306['Asn. w/ deam.'] = sum_306['N-deam-NAAF'] / sum_306['N-NAAF'] #3
sum_306['Glu. w/ deam.'] = sum_306['Q-deam-NAAF'] / sum_306['Q-NAAF'] #3


sum_306['Overall modified'] = 0.001*((sum_306['Met. w/ oxid.']*sum_306['M-NAAF']) + \
                                     (sum_306['Asn. w/ deam.']*sum_306['N-NAAF']) + \
                                     (sum_306['Glu. w/ deam.']*sum_306['Q-NAAF']) / (sum_306['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_306.to_csv("processed/NAAF-sums/NAAF-sum-306.csv")

sum_306.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
306,1043.313229,796.872011,32.841721,17.004157,29.523565,34.520154,72.05914,48.405025,20.753494,58.165775,...,7.437963,8.055625,27.389917,8890.634321,265.515326,1.0,0.233311,0.045349,0.489118,0.015597


### 4. Chaves, timepoint 24 hrs, size fraction >0.7 um (denoted as CV_T24_GD)
### T7 samples #s: 406A, 406B
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/406_CV_T24_GD_PDB
    Trocas-incubations/processed/PeaksDB/406_CV_T24_GD_DN

In [6]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_406a = pd.read_csv("processed/PeaksDN/406_CV_T24_GD_DN/406A_CV_T24_GD_DN50_naaf_totals.csv")
peaks50_406b = pd.read_csv("processed/PeaksDN/406_CV_T24_GD_DN/406B_CV_T24_GD_DN50_naaf_totals.csv")
peaksdb_406a = pd.read_csv("processed/PeaksDB/406_CV_T24_GD_PDB/406A_CV_T24_GD_PDB_naaf_totals.csv")
peaksdb_406b = pd.read_csv("processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_naaf_totals.csv")


frames = [peaks50_406a, peaks50_406b, peaksdb_406a, peaksdb_406b, ]
index = ['peaks50_406a', 'peaks50_406b', 'peaksdb_406a', 'peaksdb_406b']

# concatenate dataframes
tot_406 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_406 = tot_406.loc[:, ~tot_406.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_406a', 'peaks50_406b', 'peaksdb_406a', 'peaksdb_406b']
tot_406.insert(loc=0, column='data source', value=names)
tot_406.set_index('data source')

# sum the AAs and PTMN

index = ['406']

data = {'NAAF': tot_406['NAAF'].sum(),
        'A-NAAF': tot_406['A'].sum(),
        'C-NAAF': tot_406['C'].sum(),
        'D-NAAF': tot_406['D'].sum(),
        'E-NAAF': tot_406['E'].sum(),
        'F-NAAF': tot_406['F'].sum(),
        'G-NAAF': tot_406['G'].sum(),
        'H-NAAF': tot_406['H'].sum(),
        'I-NAAF': tot_406['I'].sum(),
        'K-NAAF': tot_406['K'].sum(),
        'L-NAAF': tot_406['L'].sum(),
        'M-NAAF': tot_406['M'].sum(),
        'N-NAAF': tot_406['N'].sum(),
        'P-NAAF': tot_406['P'].sum(),
        'Q-NAAF': tot_406['Q'].sum(),
        'R-NAAF': tot_406['R'].sum(),
        'S-NAAF': tot_406['S'].sum(),
        'T-NAAF': tot_406['T'].sum(),
        'V-NAAF': tot_406['V'].sum(),
        'W-NAAF': tot_406['W'].sum(),
        'Y-NAAF': tot_406['Y'].sum(),
        'C-carb-NAAF': tot_406['c-carb'].sum(),
        'M-oxid-NAAF': tot_406['m-oxid'].sum(),
        'N-deam-NAAF': tot_406['n-deam'].sum(),
        'Q-deam-NAAF': tot_406['q-deam'].sum()
       }

sum_406 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_406['AA-NAAF'] = sum_406['A-NAAF'] + sum_406['C-NAAF'] + sum_406['D-NAAF'] + sum_406['E-NAAF'] + \
                     sum_406['F-NAAF'] + sum_406['G-NAAF'] + sum_406['H-NAAF'] + sum_406['I-NAAF'] + \
                     sum_406['K-NAAF'] + sum_406['L-NAAF'] + sum_406['M-NAAF'] + sum_406['N-NAAF'] + \
                     sum_406['P-NAAF'] + sum_406['Q-NAAF'] + sum_406['R-NAAF'] + sum_406['S-NAAF'] + \
                     sum_406['T-NAAF'] + sum_406['V-NAAF'] + sum_406['W-NAAF'] + sum_406['Y-NAAF'] 

sum_406['AA-modifiable-NAAF'] = sum_406['M-NAAF'] + sum_406['N-NAAF'] + sum_406['Q-NAAF']
                                

sum_406['Cys. w/ carb.'] = sum_406['C-carb-NAAF'] / sum_406['C-NAAF'] #1
sum_406['Met. w/ oxid.'] = sum_406['M-oxid-NAAF'] / sum_406['M-NAAF'] #2
sum_406['Asn. w/ deam.'] = sum_406['N-deam-NAAF'] / sum_406['N-NAAF'] #3
sum_406['Glu. w/ deam.'] = sum_406['Q-deam-NAAF'] / sum_406['Q-NAAF'] #3


sum_406['Overall modified'] = 0.001*((sum_406['Met. w/ oxid.']*sum_406['M-NAAF']) + \
                                     (sum_406['Asn. w/ deam.']*sum_406['N-NAAF']) + \
                                     (sum_406['Glu. w/ deam.']*sum_406['Q-NAAF']) / (sum_406['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_406.to_csv("processed/NAAF-sums/NAAF-sum-406.csv")

sum_406.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
406,740.555985,572.361371,12.371238,15.061769,14.104245,11.772214,29.864431,18.597357,45.643743,21.258306,...,3.983931,3.393869,1.710341,6164.049195,124.338606,1.0,0.234577,0.034088,0.219486,0.007392
